In [34]:
import aup
from keplergl import KeplerGl
import geopandas as gpd
import hex_config
import pandas as pd

In [35]:
cities = ['Aguascalientes','Cuernavaca','Guadalajara','Laguna','Merida',
          'Mexicali','Mexico','Monterrey','Puebla','Querétaro','Saltillo',
         'SLP','Toluca']

In [36]:
def load_data(df, c):
    aup.log(f"{c} loading data")
    # Creates empty GeoDataFrame to store specified municipality polygons
    mun_gdf = gpd.GeoDataFrame()
    # ageb_gdf = gpd.GeoDataFrame()
    hex_bins = gpd.GeoDataFrame()
    year = 2020
    mpos_folder = f"mpos_{year}"
    for i in range(len(df.loc["mpos", c])):
        # Extracts specific municipality code
        m = df.loc["mpos", c][i]
        # Downloads municipality polygon according to code
        query = f"SELECT * FROM marco.{mpos_folder} WHERE \"CVEGEO\" LIKE '{m}'"
        mun_gdf = mun_gdf.append(aup.gdf_from_query(
            query, geometry_col="geometry"))
        # Creates query to download hex bins
        query = f"SELECT * FROM processed.hex_bins_index_{year} WHERE \"CVEGEO\" LIKE '{m}%%'"
        hex_bins = hex_bins.append(
            aup.gdf_from_query(query, geometry_col="geometry"))
    return hex_bins


In [37]:
df = pd.read_json("../scripts/Metropolis_CVE.json")


In [38]:
hex_bins = load_data(df, cities[0])

In [39]:
#column = 'idx_accessibility'
column = 'dist_hospitales'
hex_bins[column] = hex_bins[column].astype(float)

if column == "idx_accessibility":
    config = hex_config.config_idx
    hex_bins[column] = hex_bins[column].round(2)
    bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
    labels = ['0-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1']
    hex_bins[f'bins_{column}'] = pd.cut(
        hex_bins[column], bins=bins, labels=labels, include_lowest=True)
    config["config"]["visState"]["layers"][0]["visualChannels"]["colorField"][
        "name"
    ] = f'bins_{column}'
    _name = 'Índice accesibilidad'
    hex_bins[_name] = hex_bins[column].astype(str)
    config["config"]["visState"]["interactionConfig"]["tooltip"]["fieldsToShow"][
        "data"
    ][0]["name"] = _name
else:
    config = hex_config.config
    config["config"]["visState"]["layers"][0]["visualChannels"]["colorField"][
        "name"
    ] = column
    _name = column.replace('dist_', 'Distancia a ')
    hex_bins[_name] = hex_bins[column].round(2).astype(str) + ' metros'
    _name = column.replace('dist_', 'Distancia a ')
    config["config"]["visState"]["interactionConfig"]["tooltip"]["fieldsToShow"][
        "data"
    ][0]["name"] = _name
hex_bins["d"] = 0
longitude = hex_bins.dissolve(by="d").geometry.centroid.x
latitude = hex_bins.dissolve(by="d").geometry.centroid.y
config["config"]["mapState"]["latitude"] = latitude[0]
config["config"]["mapState"]["longitude"] = longitude[0]
map_city = KeplerGl(height=800, data={"data": hex_bins}, config=config)
#map_city.save_to_html(data ={"data": hex_bins}, file_name="temp.html", read_only=True, config = config)

<ipython-input-39-e7ac13e66adf>:32: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  longitude = hex_bins.dissolve(by="d").geometry.centroid.x


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


<ipython-input-39-e7ac13e66adf>:33: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  latitude = hex_bins.dissolve(by="d").geometry.centroid.y
/Users/natera/miniconda3/envs/ox/lib/python3.8/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [40]:
map_city

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'jsx1yd', 'type': '…

In [41]:
config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'jsx1yd',
     'type': 'geojson',
     'config': {'dataId': 'data',
      'label': 'data',
      'color': [231, 159, 213],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.35,
       'strokeOpacity': 0.05,
       'thickness': 0.5,
       'strokeColor': [28, 27, 27],
       'colorRange': {'name': 'Custom Palette',
        'type': 'custom',
        'category': 'Custom',
        'colors': ['#FAE300', '#FD7900', '#CF1750', '#7A0DA6', '#2C51BE']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'stroked': True,
       'filled

In [43]:
map_city.config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'jsx1yd',
     'type': 'geojson',
     'config': {'dataId': 'data',
      'label': 'data',
      'color': [231, 159, 213],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.35,
       'strokeOpacity': 0.05,
       'thickness': 0.5,
       'strokeColor': [28, 27, 27],
       'colorRange': {'name': 'Custom Palette',
        'type': 'custom',
        'category': 'Custom',
        'colors': ['#FAE300', '#FD7900', '#CF1750', '#7A0DA6', '#2C51BE']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'stroked': True,
       'filled

In [32]:
map_city.save_to_html(data ={"data": hex_bins}, file_name="temp.html", read_only=True, config = config)

Map saved to temp.html!


/Users/natera/miniconda3/envs/ox/lib/python3.8/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [11]:
with open('hex_config_2.py', 'w') as f:
   f.write('config = {}'.format(map_city.config))

In [ ]:
map_1 = KeplerGl(height=800, config=config)
m_m = 83.3
for city in cities:
    gdf = gpd.read_file(f'../data/processed/{city}_hex_bins.geojson')
    gdf['dist_promedio'] = (gdf['dist_farmacias']+gdf['dist_hospitales']+gdf['dist_farmacias'])/3
    gdf['min_farmacias'] = gdf['dist_farmacias']/m_m
    gdf['min_supermercados'] = gdf['dist_supermercados']/m_m
    gdf['min_hospitales'] = gdf['dist_hospitales']/m_m
    gdf['tiempo_promedio'] = (gdf['min_hospitales']+gdf['min_supermercados']+gdf['min_farmacias'])/3
    map_1.add_data(gdf[gdf['dist_promedio']>0], city)
    print(city)
map_1

In [ ]:
for i in range(13):
    map_1.config['config']['visState']['layers'][i]['config']['visConfig']['opacity']=0.1

In [ ]:
# Save map_1 config to a file
with open(f'config_kepler_plot.py', 'w') as f:
   f.write('config = {}'.format(map_1.config))

In [ ]:
# load the config
%run config_kepler_plot.py

In [ ]:
map_1.save_to_html(file_name=f'../output/html/Mexico_accesibilidad.html', config=config,read_only=False)

In [ ]:
config['config']['visState']['layers'][12]

In [31]:
config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'jsx1yd',
     'type': 'geojson',
     'config': {'dataId': 'data',
      'label': 'data',
      'color': [231, 159, 213],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.35,
       'strokeOpacity': 0.05,
       'thickness': 0.5,
       'strokeColor': [28, 27, 27],
       'colorRange': {'name': 'Custom Palette',
        'type': 'custom',
        'category': 'Custom',
        'colors': ['#FAE300', '#FD7900', '#CF1750', '#7A0DA6', '#2C51BE']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'stroked': True,
       'filled